# **CS 5361 Machine Learning**
**Final Project: Machine Learning from Scratch**

**Author:**  Robert Alvarez<br>

**Last modified:** Nov 18th  <br>

Modules use for project development

In [ ]:
import numpy as np
import pandas as pd

import numpy.typing as npt
from typing import Tuple
from nptyping import NDArray, Shape, Float, Integer, Number, Obj

from sklearn.model_selection import train_test_split
from abc import ABC, abstractmethod
from time import time
from math import pi
from google.colab import files

Upload data sets.

In [ ]:
upload = files.upload()

Saving Datasets.zip to Datasets.zip


In [ ]:
!unzip -q Datasets.zip

In [ ]:
!ls

Datasets.zip	 gpu_running_time.csv  particles_y.npy
doh_dataset.csv  __MACOSX	       Pecan.txt
gamma04.txt	 particles_X.npy       sample_data


## Numpy Types

In [ ]:
NPArrayNxM = NDArray[Shape["N, M"], Obj]

NumNPArrayNxM = NDArray[Shape["N, M"], Number]
NumNPArray = NDArray[Shape["N"], Number]

FloatNPArrayNxM = NDArray[Shape["N, M"], Float]
FloatNPArrayNxN = NDArray[Shape["N, N"], Float]
FloatNPArray = NDArray[Shape["N"], Float]

IntNPArrayNxM = NDArray[Shape["N, M"], Integer]
IntNPArray = NDArray[Shape["N"], Integer]

ArrayLike = npt.ArrayLike

## **Performance Metrics**


Precision$(y,p) = \frac{TP}{TP+FP}$

In [ ]:
def precision(y: NumNPArray, p: NumNPArray) -> np.float64:
  """
  Calculate precision between ground truth and estimates.

  Args:
      y (NumNPArray): Ground truth (correct) target values.
      p (NumNPArray): Estimated targets as returned by a classifier.

  Returns:
      np.float64: Ratio between the True Positives and all the points that are classified as Positives.
  """
  return np.sum(y*p)/np.sum(p)

Recall$(y,p) = \frac{TP}{TP+FN}$

In [ ]:
def recall(y: NumNPArray, p: NumNPArray) -> np.float64:
  """
  Calculate recall between ground truth and estimates.

  Args:
      y (NumNPArray): Ground truth (correct) target values.
      p (NumNPArray): Estimated targets as returned by a classifier.

  Returns:
      np.float64: Measure of our model correctly identifying True Positives.
  """
  return np.sum(y*p)/np.sum(y)

$
    F1 = \frac{2*Precision(y,p)*Recall(y,p)}{Precision(y,p)+Recall(y,p)}
$

In [ ]:
def f1(y: NumNPArray, p: NumNPArray) -> np.float64:
  """
  Calculate f1 score between ground truth and estimates.

  Args:
      y (NumNPArray): Ground truth (correct) target values.
      p (NumNPArray): Estimated targets as returned by a classifier.

  Returns:
      np.float64: Harmonic mean of the Precision and Recall.
  """
  pr = precision(y,p)
  r = recall(y,p)
  return 2*pr*r/(pr+r)

Accuracy$(y,p) = \frac{TP+TN}{TP+FN+TN+FP}$

In [ ]:
def accuracy(y: NumNPArray, p: NumNPArray) -> np.float64:
  """
  Calculate accuracy between ground truth and estimates.

  Args:
      y (NumNPArray): Ground truth (correct) target values.
      p (NumNPArray): Estimated targets as returned by a classifier.

  Returns:
      np.float64: Fraction of correctly classified samples.
  """
  return np.sum(y==p)/len(y)

Specificity$(y,p) = \frac{TN}{TN+FP}$

In [ ]:
def specificity(y: NumNPArray, p: NumNPArray) -> np.float64:
  """
  Calculate specificity between ground truth and estimates.

  Args:
      y (NumNPArray): Ground truth (correct) target values.
      p (NumNPArray): Estimated targets as returned by a classifier.

  Returns:
      np.float64: Recall of the negative class.
  """
  return np.sum((1-y)*(1-p))/np.sum(y==p)

Confusion matrix:

In [ ]:
def confusion_matrix(y: NumNPArray, p: NumNPArray) -> list[list[int]]:
  """
  Compute confusion matrix to evaluate the accuracy of a classification.

  Args:
      y (NumNPArray): Ground truth (correct) target values.
      p (NumNPArray): Estimated targets as returned by a classifier.

  Returns:
      list[list[int]]: Confusion matrix whose i-th row and j-th column entry indicates the number of samples with true label being i-th class and predicted label being j-th class.
  """
  y_u = np.unique(y)
  m = []
  for i in y_u:
    p_i = p[y==i]
    arr = []
    for j in y_u:
      arr.append(np.sum(p_i==j))
    m.append(arr)
  return m

\begin{equation}
MAE=\frac{1}{n}\sum_{i=0}^{n}|Y_i-\hat{Y_i}|
\end{equation}

In [ ]:
def MAE(y: NumNPArray, p: NumNPArray) -> np.float64:
  """
  Calculate Mean Absolute Error between ground truth and estimates.

  Args:
      y (NumNPArray): Ground truth (correct) target values.
      p (NumNPArray): Estimated targets as returned by a classifier.

  Returns:
      np.float64: Average of all output errors is returned.
  """
  return np.sum(np.absolute(y-p))/y.shape[0]

\begin{equation}
MSE=\frac{1}{n}\sum_{i=0}^{n}\big(Y_i-\hat{Y_i}\big)^2
\end{equation}

In [ ]:
def MSE(y: NumNPArray, p: NumNPArray) -> np.float64:
  """
  Calculate Mean Square Error between ground truth and estimates.

  Args:
      y (NumNPArray): Ground truth (correct) target values.
      p (NumNPArray): Estimated targets as returned by a classifier.

  Returns:
      np.float64: Average of all output square errors is returned.
  """
  return np.sum((y-p)**2)/y.shape[0]

## **Utils**


The funciton to_categorical(y) recieve a vector y of size n, it find the k unique values in y from 0 to k-1 and it makes a n-by-k matrix where $y_{i,j} = 1$ if $y_i = j$ and 0 otherwise.

In [ ]:
def to_categorical(y: IntNPArray) -> IntNPArray:
  y_vals = np.unique(y)
  cat_y = np.empty(shape=(y.shape[0],y_vals.shape[0]))
  for i,y_val in enumerate(y_vals):
    cat_y[:,i] = (y==y_val).astype(int)
  return cat_y

The function standardize(x) calculates $\frac{x-\mu}{\sigma}$ for each column in x.

In [ ]:
def standardize(x: FloatNPArrayNxM) -> FloatNPArrayNxM:
  for i in range(x.shape[1]):
    x[:,i] = (x[:,i] - np.mean(x[:,i]))/np.std(x[:,i])

LebelEncoder is a class that label targest with values from 0 to k-1, where k is the number unique values. Fit find the k unique values on the data, transform change the targest values to the respective encode number and inverse_transform converts the numbers to the map values.

In [ ]:
class LabelEncoder:
  """
  Class use to encode y values from 0 to n_classes-1, where n_classes is the
  number of unique y values.
  ...
  Attributes
  ----------
  classes_ : numpy array of shape (n_classes,)
    classes_ is a n_classes entries numpy array where each entry have a unique
    y value

   Methods
  ---------
  fit(data)
    Find the k unique values on the data and save them in classes_ attribute.
  transform(data)
    Transform change the targest values to the respective encode number.
  inverse_transform(data)
    Reverse the transformer by mapping the numbers from 0 to n_classes to its
    original value.
  """
  # Constructor
  def __init__(self) -> None:
    self.classes_ = np.empty((0))

  def fit(self, data: ArrayLike) -> None:
    """
    Find the k unique values on the data and save them in classes_ attribute.

    Args:
        data (ArrayLike): _description_
    """
    self.classes_ = np.unique(data)

  def fit_transform(self, data: ArrayLike) -> IntNPArray:
    #Get the classes if they haven't being set
    if self.classes_.shape[0] == 0:
      self.fit(data)
    return self.transform(data)

  def transform(self, data: ArrayLike) -> IntNPArray:
    """
    It use classes_ to encode each data value to the respective index in the array.

    Args:
        data (ArrayLike): _description_

    Returns:
        IntNPArray: _description_
    """
    if isinstance(data, list):
      data = np.array(data, self.classes_.dtype)
    temp = np.empty(data.shape,np.int32)
    for i,c in enumerate(self.classes_):
      temp[c==data] = i
    return temp

  def inverse_transform(self, data: IntNPArray) -> ArrayLike:
    """ Replace each data values to what is in classes_ by using each data value as an index to classes_.

    Args:
        data (IntNPArray): _description_

    Returns:
        ArrayLike: _description_
    """
    if isinstance(data, list):
      shape = len(data)
    else:
      shape = data.shape
    temp = np.empty(shape,self.classes_.dtype)
    for i,c in enumerate(self.classes_):
      i = np.int32(i)
      temp[i==data] = c
    return temp

$relu(x) = max(0,x)$

In [ ]:
def relu(x: FloatNPArray) -> FloatNPArray:
  """
  Return the maximum number between 0 and x for each value of x.

  Args:
      x (FloatNPArray): _description_

  Returns:
      FloatNPArray: Maximum number between 0 and x for each value of x.
  """
  return np.maximum(np.array([[0]]),x)

$sigmoid(z) = \frac{1}{1+exp(-z)}$

In [ ]:
def sigmoid(x: FloatNPArray) -> FloatNPArray:
  """
  Returns 1/(1+exp(-z)) for each value of z.

  Args:
      x (FloatNPArray): _description_

  Returns:
      FloatNPArray: 1/(1+exp(-z)) for each value of z.
  """
  return 1./(1.+np.exp(-z))

## **Data set processsing**


In [ ]:
#Binary classification
def process_gamma_dataset():
  #Read file information
  infile = open("gamma04.txt","r")
  x, y  = [], []
  for line in infile:
    y.append(int(line[-2:-1] == 'g'))
    x.append(np.fromstring(line[:-2], dtype=float, sep=','))
  infile.close()

  #Make dependent and independent numpy arrays
  x = np.array(x).astype(np.float32)
  y = np.array(y)

  #Split the data
  return train_test_split(x, y, test_size=0.2, random_state=4361)

#Regression
def process_gpu_running_time():
  df = pd.read_csv('gpu_running_time.csv')
  data = df.to_numpy()
  X = data[:,:15]
  y = np.mean(data[:,15:],axis=1)
  return train_test_split(X, y, test_size=0.3, random_state=4361)

#Binary classification
def process_doh():
  # Drop rows that have a NaN in them
  df = pd.read_csv("doh_dataset.csv", compression='gzip').dropna()
  # Remove non-float data as that will mess up the student's code
  df = df.drop(['TimeStamp', 'SourceIP', 'DestinationIP'])
  # Extract the labels from the dataframe and encode them to integers
  df_labels = df.pop('Label')
  label_encoder = LabelEncoder()
  df_labels = label_encoder.fit_transform(df_labels)

  # Prepare arrays to split into training and testing sets
  x_features = df.values
  y_labels = np.array(df_labels).T

  # Split into training (70%) and testing (30%)
  return train_test_split(x_features, y_labels, train_size=0.7, random_state=1738, shuffle=True)

#Multi-classification 
def mnist():
  import tensorflow as tf
  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
  x_train = np.float32(x_train.reshape(x_train.shape[0],-1)/255)
  x_test = np.float32(x_test.reshape(x_test.shape[0],-1)/255)
  return x_train,x_test,y_train,y_test

#Regression
def process_particles():
  PX = np.load('particles_X.npy')
  Py = np.load('particles_y.npy')
  return train_test_split(PX, Py, test_size=0.2,  random_state=4361)

In [ ]:
#Save mnist dataset
mnist_x_train, mnist_x_test, mnist_y_train, mnist_y_test = mnist()

## **Nearest Neighbor**


NearestNeighbor is a super class use by NearestNeighborClassifier and NearestNeighborRegressor.

We denote the set of k nearest neighbor of $\mathbf{x}$ as $S_\mathbf{x}$. Where $S_\mathbf{x}$ is defined as $S_\mathbf{x} \subseteq D$ s.t. $|S_\mathbf{x}|=k$ and $\forall (\mathbf{x}',y') \in D \backslash S_\mathbf{x}$,
\begin{equation}
  dist(\mathbf{x},\mathbf{x}') \ge \max_{{(\mathbf{x}'',y'') \in S_\mathbf{x}}} dist(\mathbf{x},\mathbf{x}'')
\end{equation}

In [ ]:
class NearestNeighbor(ABC):
  """
  Abstract class use by KNeighborsClassifier and KNeighborsRegressor.
  ...
  Atributes
  ---------
  k : int
    Number of nearest neighbors to be find when predicting
  weighted : boolean
    True for weighted and false for unweighted distance calculations.

   Methods
  ---------
  fit(x_train, y_train, regression=False)
    Save x_train and y_train regardless of regression boolean value. If regression=False
    the unique y values are save in classes_ as another attribute of the class.
  get_closest_k(x_test)
    Calculate and return the k nearest neighbors for each x_test base on the x_train
    values saved when calling fit.
  """
  # Constructor
  def __init__(self, k: int = 1, weighted: bool = False) -> None:
    """
    Save parameters k=1 and weighted=False if no parameter are specified.

    Args:
        k (int, optional): Number of nearest neighbor to be consider for predictions. Defaults to 1.
        weighted (bool, optional): True for weighted and false for unweighted distance calculations. Defaults to False.
    """
    self.k = k
    self.weighted = weighted

  def fit(self, x_train: NumNPArrayNxM, y_train: ArrayLike, regression: bool = False) -> None:
    """
    Save x_train and y_train regardless of regression boolean value. If regression=False
    the unique y values are save in classes_ as another attribute of the class.

    Args:
        x_train (NumNPArrayNxM): _description_
        y_train (ArrayLike): _description_
        regression (bool, optional): _description_. Defaults to False.
    """
    self.x_train = x_train
    self.y_train = y_train
    if not regression:
      self.classes_ = np.unique(y_train)

  def get_closest_k(self, x_test: NumNPArrayNxM) -> NumNPArray:
    """
    Calculate and return the k nearest neighbors for each x_test base on the x_train
    values saved when calling fit.

    Args:
        x_test (NumNPArrayNxM): _description_

    Returns:
        NumNPArray: _description_
    """
    # Improved version using the fact that (a-b)**2 = a**2 - 2ab + b**2
    dist1 = np.sum(x_test**2,axis=1,keepdims=True)
    dist2 = -2.*np.matmul(x_test,self.x_train.T)
    # This part is not really necessary, since it does not depend on x_test
    dist3 = np.sum(self.x_train.T**2,axis=0,keepdims=True)
    dist = dist1+dist2+dist3
 
    # Distance matrix is created, get the k closest elements
    return np.argpartition(dist, kth=self.k, axis=-1)[:,:self.k], dist

  @abstractmethod
  def predict(self, x_test):
    pass

### **Classifier**

With the same definition for $S_x$ we can define the classifier p() for unweighted distance as:
\begin{equation}
  p(\mathbf{x}) = mode(\{y'' : (\mathbf{x}'',y'') \in S_\mathbf{x}\})
\end{equation}
and for weighted distance as:
\begin{equation}
  p(\mathbf{x}) = \underset{c \in C}{\mathrm{argmin}} \sum_{{(\mathbf{x}'',y'') \in S_\mathbf{x}}} \frac{1}{dist(\mathbf{x},\mathbf{x}_i'')} \delta_{c,y_i''}
\end{equation}
where $C$ is the set off all possible $y \in D$.

In [ ]:
class KNeighborsClassifier(NearestNeighbor):
  """
  Extension of NearestNeighbor to be use as a classifier. Classification is done
  by chossing the most common target value from the k nearest neighbors.
  """
  # Predict class of test data
  def predict(self, x_test: NumNPArrayNxM) -> NumNPArray:
    """
    Get k nearest neighbors by calling get_closest_k from the super class with
    x_test as parameter. For unweighted distance, use those indeces to the k closes
    elements to get the mode out of the y value of those indeces. For weighted
    distance, use those indeces to get the target values and predict the maximum
    argument of the class with largest number where the number of each class
    is calculates by adding 1/dist(x,x'') to the class where x'' belongs to.

    Args:
        x_test (NumNPArrayNxM): _description_

    Returns:
        NumNPArray: _description_
    """
    # Distance matrix is created, get the k closest elements
    minIdxs, dist = super().get_closest_k(x_test)

    # Build weight array with the votes and choose the one with the highest one to predict
    possibles = np.zeros((dist.shape[0],self.classes_.shape[0]))
    for irow,rowIdx in enumerate(minIdxs):
      for idx in rowIdx:
        if self.weighted:
          possibles[irow,self.y_train[idx]] += 1/dist[irow,idx]
        else:
          possibles[irow,self.y_train[idx]] += 1

    return self.classes_[np.argmax(possibles, axis=1)]

Classification test

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNNC

if __name__ == '__main__':
  x_train,x_test,y_train,y_test = mnist_x_train, mnist_x_test, mnist_y_train, mnist_y_test

  for weighted in [False,True]:
    weight = 'distance' if weighted else 'uniform'
    for k in range(1,16,2):
      print('k =',k,'weighted =',weighted)
      print("My model:")
      t0 = time()
      model = KNeighborsClassifier(k=k, weighted=weighted)
      model.fit(x_train, y_train)
      pred = model.predict(x_test)
      print("\tElapse time = {:.5f}".format(time() - t0))
      print('\tAccuracy = {:6.4f}'.format(accuracy(y_test, pred)))

      print("SKLEARN model:")
      t0 = time()
      model = KNNC(n_neighbors=k, weights=weight)
      model.fit(x_train, y_train)
      pred = model.predict(x_test)
      print("\tElapse time = {:.5f}".format(time() - t0))
      print('\tAccuracy = {:6.4f}\n'.format(accuracy(y_test, pred)))

k = 1 weighted = False
My model:
	Elapse time = 25.42723
	Accuracy = 0.9691
SKLEARN model:
	Elapse time = 40.06176
	Accuracy = 0.9691

k = 3 weighted = False
My model:
	Elapse time = 27.20054
	Accuracy = 0.9705
SKLEARN model:
	Elapse time = 40.52692
	Accuracy = 0.9705

k = 5 weighted = False
My model:
	Elapse time = 27.93530
	Accuracy = 0.9688
SKLEARN model:
	Elapse time = 40.48159
	Accuracy = 0.9688

k = 7 weighted = False
My model:
	Elapse time = 27.73766
	Accuracy = 0.9694
SKLEARN model:
	Elapse time = 40.21772
	Accuracy = 0.9694

k = 9 weighted = False
My model:
	Elapse time = 27.98494
	Accuracy = 0.9659
SKLEARN model:
	Elapse time = 41.28986
	Accuracy = 0.9659

k = 11 weighted = False
My model:
	Elapse time = 36.57435
	Accuracy = 0.9668
SKLEARN model:
	Elapse time = 57.19568
	Accuracy = 0.9668

k = 13 weighted = False
My model:
	Elapse time = 35.17777
	Accuracy = 0.9653
SKLEARN model:
	Elapse time = 40.59555
	Accuracy = 0.9653

k = 15 weighted = False
My model:
	Elapse time = 29.5

### **Regressor**

With the same definition for $S_x$ we can define our regressor $\hat{y}$ as:
\begin{equation}
  \hat{y}(\mathbf{x}) = \frac{1}{k} \sum_{i=1}^k \frac{y_i''}{w_i}
\end{equation}
where $w_i = 1$ for unweigthed distance and $w_i = dist(\mathbf{x},\mathbf{x}_i'')$ for weighted distance.

In [ ]:
class KNeighborsRegressor(NearestNeighbor):
  """
  Extension of NearestNeighbor to be use as a regressor. Regression is done by
  getting the k closest point at taking the average of each dimension.
  """
  def predict(self, x_test: NumNPArrayNxM) -> NumNPArray:
    """
    Prediction is done with the closest k points by taking the average of each
    dimension for unweighted distance and the average with the weight of
    1/dist(x,x'') for weighted distance.

    Args:
        x_test (NumNPArrayNxM): _description_

    Returns:
        NumNPArray: _description_
    """
    # Distance matrix is created, get the k closest elements
    minIdxs, dist = super().get_closest_k(x_test)

    if self.weighted:
      weights = np.array([1/dist[irow,rowIdx] for irow,rowIdx in enumerate(minIdxs)])
      ans = np.average(self.y_train[minIdxs],axis=1,weights=weights)
    else:
      ans = np.mean(self.y_train[minIdxs],axis=1)
    return ans

Regressor test

In [ ]:
from sklearn.neighbors import KNeighborsRegressor as KNNR

print("Using Pecan.txt for regression:")
#Read data
df = pd.read_csv("Pecan.txt", delimiter="\t")

#Remove first and last column
X = df.values[:, range(1, len(df.columns)-1)]
Y = df.values[:, len(df.columns)-1]
newData = np.array([[120,5,80], [20,40,15]])

for weighted in [False,True]:
  weight = 'distance' if weighted else 'uniform'
  for k in range(1,6,2):
    print('k =',k)
    print("My model:")
    t0 = time()
    model = KNeighborsRegressor(k=k,weighted=weighted)
    model.fit(X,Y)
    print("\tPrediction = ", model.predict(newData))
    print("\tElapse time = {:.5f}".format(time() - t0))

    print("SKLEARN model:")
    t0 = time()
    model = KNNR(n_neighbors=k,weights=weight)
    model.fit(X,Y)
    print("\tPrediction = ", model.predict(newData))
    print("\tElapse time = {:.5f}\n".format(time() - t0))

Using Pecan.txt for regression:
k = 1
My model:
	Prediction =  [543.79796 310.     ]
	Elapse time = 0.01567
SKLEARN model:
	Prediction =  [543.79796 310.     ]
	Elapse time = 0.00375

k = 3
My model:
	Prediction =  [534.9925781  313.71552503]
	Elapse time = 0.00113
SKLEARN model:
	Prediction =  [534.9925781  313.71552503]
	Elapse time = 0.00281

k = 5
My model:
	Prediction =  [530.5904805  319.46238006]
	Elapse time = 0.00098
SKLEARN model:
	Prediction =  [530.5904805  319.46238006]
	Elapse time = 0.00233

k = 1
My model:
	Prediction =  [543.79796 310.     ]
	Elapse time = 0.00126
SKLEARN model:
	Prediction =  [543.79796 310.     ]
	Elapse time = 0.00242

k = 3
My model:
	Prediction =  [535.60945781 313.24697075]
	Elapse time = 0.00110
SKLEARN model:
	Prediction =  [535.29663986 313.4814209 ]
	Elapse time = 0.00223

k = 5
My model:
	Prediction =  [531.4747662  317.61443499]
	Elapse time = 0.00103
SKLEARN model:
	Prediction =  [531.02382964 318.52821935]
	Elapse time = 0.00245



## **Linear Regression**


Linear regression model tries to approximate $y$ as
\begin{equation}
  \hat{y} = \beta_0 + \sum_{j=1}^p \mathbf{X}_j\beta_j
\end{equation}

giving a reduse sum of squares (RSS) of
\begin{align}
  RSS(\beta) &= \sum_{i=1}^N (y_i - \hat{y_i})^2 \\
             &= \sum_{i=1}^N \big(y_i -\beta_0 - \sum_{j=1}^p x_{ij}\beta_j\big)^2
\end{align}

Some calculus and matrix properties use:
- $\nabla_x (f(x)g(x)) = g(x)\nabla_xf(x) + f(x)\nabla_xg(x) \text{ where } x \in \mathbb{R}^n \text{ and } f(x),g(x): \mathbb{R}^n \rightarrow \mathbb{R} $
- $a,b \in \mathbb{R}^n, a^Tb = b^Ta \text{ because } a^Tb \text{ and } b^Ta \text{ are scalar values.} $
- $\beta \in \mathbb{R}^n, \frac{\partial}{\partial \beta} \beta^T = \delta_{i,j} = \mathbf{I}_n$

By appending a column of 1's as the first column of $\mathbf{x}$ we can rewrite $RSS(\beta)$ as
\begin{align}
  RSS(\beta) &= (\mathbf{y} - \mathbf{X}\beta)^T(\mathbf{y} - \mathbf{X}\beta) \\
  &= (\mathbf{y}^T - \beta^T\mathbf{X}^T)(\mathbf{y} - \mathbf{X}\beta) \\
  &= \mathbf{y}^T\mathbf{y} - \mathbf{y}^T\mathbf{X}\beta - \beta^T\mathbf{X}^T\mathbf{y} + \beta^T\mathbf{X}^T\mathbf{X}\beta \\
  &= \mathbf{y}^T\mathbf{y} - 2\mathbf{y}^T\mathbf{X}\beta + \beta^T\mathbf{X}^T\mathbf{X}\beta
\end{align}

To minimize this function, we would take the first derivative of $RSS$.
\begin{align}
  \frac{\partial RSS}{\partial \beta} &= \frac{\partial}{\partial \beta}(\mathbf{y}^T\mathbf{y}) - 2\frac{\partial}{\partial \beta}(\beta^T\mathbf{X}^T\mathbf{y}) + \frac{\partial}{\partial \beta}(\beta^T\mathbf{X}^T\mathbf{X}\beta) \\
  &= 0 - 2\mathbf{I}\mathbf{X}^T\mathbf{y} + \frac{\partial}{\partial \beta}(\beta^T)\mathbf{X}^T\mathbf{X}\beta + \beta^T\mathbf{X}^T\mathbf{X} \big(\frac{\partial}{\partial \beta}\beta \big) \\
  &= -2\mathbf{X}^T\mathbf{y} + \mathbf{I}\mathbf{X}^T\mathbf{X}\beta + \frac{\partial}{\partial \beta}(\beta^T) (\beta^T\mathbf{X}^T\mathbf{X})^T \\
  &= -2\mathbf{X}^T\mathbf{y} + \mathbf{X}^T\mathbf{X}\beta + \mathbf{X}^T\mathbf{X}\beta \\
  &= -2\mathbf{X}^T\mathbf{y} + 2\mathbf{X}^T\mathbf{X}\beta \\
  &= -2\mathbf{X}^T(\mathbf{y} - \mathbf{X}\beta)
\end{align}

For the second derivative we get
\begin{equation}
  \frac{\partial^2 RSS}{\partial\beta \partial\beta^T} = 2\mathbf{X}^T\mathbf{X}
\end{equation}

Assuming that $\mathbf{X}$ has full column rank, and hence $\mathbf{X}^T\mathbf{X}$ is positive definite, we set the first partial derivative to zero and solve for $\beta$.
\begin{equation}
  \frac{\partial RSS}{\partial \beta} = -2\mathbf{X}^T(\mathbf{y} - \mathbf{X}\beta) = 0 \\
  \mathbf{X}^T\mathbf{y} - \mathbf{X}^T\mathbf{X}\beta = 0 \\
  \mathbf{X}^T\mathbf{X}\beta = \mathbf{X}^T\mathbf{y} \\
  (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{X}\beta = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y} \\
  \hat{\beta} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}
\end{equation}

We can now calcualte $\hat{y}$ as
\begin{equation}
  \hat{y} = \mathbf{X}\hat{\beta} = \mathbf{X}(\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}
\end{equation}

In [ ]:
class LinearRegression:
  """
  Module use for regression and classification. If used for classification,
  a one-hot representation neeeds to be pass.
  ...
  Atributes
  ---------
  self.intercept_ : shape=(n_classes, )
    Real value numbers that represent the bias of each class.
  self.coef_ : shape=(n_classes, n_attributes, )
    Real value numbers that represent the weights of each class.

   Methods
  ---------
  fit(x_train, y_train)
    Calcualte beta values as (X^T X)^-1 X^T Y and save them in self.intercept_
    and self.coef_.
  predict(x_test)
    Use the self.intercept_ and self.coef_ to calculate the predicted values as
    X(X^T X)^-1 X^T Y.
  """
  def __init__(self) -> None:
    pass

  #(X.TX)^-1 X.T y
  def fit(self, x_train: NumNPArrayNxM, y_train: ArrayLike) -> None:
    """
    Use (X^T X)^-1 X^T Y to calculate the beta values.
    """
    #Add column of ones to also calculate beta_0
    X = np.hstack((np.ones(shape=(x_train.shape[0],1)),x_train))
    ny_cols = 1 if len(y_train.shape) == 1 else y_train.shape[1]
    y = np.reshape(y_train,(-1,ny_cols))

    #Calculate beta coefficients
    temp = np.linalg.pinv(np.matmul(X.T, X)) #(X.TX)^-1
    temp = np.matmul(temp,X.T)

    #Calculate linear regression for each column in y
    all_betas = np.matmul(temp,y)
    self.intercept_ = all_betas[0]
    self.coef_ = all_betas[1:].T

    if ny_cols == 1:
      self.coef_ = np.reshape(self.coef_,(-1))
      self.intercept_ = np.reshape(self.intercept_,(-1))

  def predict(self, x_test: NumNPArrayNxM) -> NumNPArray:
    """
    Calculate predicted values as X(X^T X)^-1 X^T Y.

    Args:
        x_test (NumNPArrayNxM): _description_

    Returns:
        NumNPArray: _description_
    """
    ny_cols = self.intercept_.shape[0]
    if ny_cols == 1:
      pred = np.matmul(x_test,self.coef_) + self.intercept_
    else:
      pred = np.matmul(x_test,self.coef_.T) + self.intercept_
    return pred

Classification test

In [ ]:
from sklearn.linear_model import LinearRegression as LR

In [ ]:
print("\nUsing mnist for classification:")
x_train,x_test,y_train,y_test = mnist_x_train, mnist_x_test, mnist_y_train, mnist_y_test
y_train_oh = to_categorical(y_train)

print("My model:")
t0 = time()
model = LinearRegression()
model.fit(x_train,y_train_oh)
pred = np.argmax(model.predict(x_test),axis=1)
print("Elapse time = {:.5f}".format(time() - t0))
print("accuracy: {:.5f}".format(accuracy(y_test, pred)))

print("\nSKLEARN model:")
t0 = time()
model = LR()
model.fit(x_train,y_train_oh)
pred = np.argmax(model.predict(x_test),axis=1)
print("Elapse time = {:.5f}".format(time() - t0))
print("accuracy: {:.5f}".format(accuracy(y_test, pred)))


Using mnist for classification:
My model:
Elapse time = 6.32316
accuracy: 0.86030

SKLEARN model:
Elapse time = 4.82961
accuracy: 0.86030


Regression test

In [ ]:
print("\nUsing process_gpu_running_time for regression:")
x_train,x_test,y_train,y_test = process_gpu_running_time()

print("My model:")
t0 = time()
model = LinearRegression()
model.fit(x_train,y_train)
pred = model.predict(x_test)
print("Elapse time = {:.5f}".format(time() - t0))
print("MSE = ", MSE(y_test, pred))

print("\nSKLEARN model:")
t0 = time()
model = LR()
model.fit(x_train,y_train)
pred = model.predict(x_test)
print("Elapse time = {:.5f}".format(time() - t0))
print("MSE = ", MSE(y_test, pred))


Using process_gpu_running_time for regression:
My model:
Elapse time = 0.05151
MSE =  14.287121532509255

SKLEARN model:
Elapse time = 0.26056
MSE =  14.287121532508602


## **Logistic Regression**


Logistic regression model have the form
\begin{align}
  log\frac{Pr(G=1|X=x)}{Pr(G=K|X=x)} &= \beta_{10} + \beta_1^Tx \\
  log\frac{Pr(G=2|X=x)}{Pr(G=K|X=x)} &= \beta_{20} + \beta_2^Tx \\
  &... \\
  log\frac{Pr(G=K-1|X=x)}{Pr(G=K|X=x)} &= \beta_{(K-1)0} + \beta_{K-1}^Tx
\end{align}

We can solve for any probability from k=1 to K-1 and get the following equation
\begin{equation}
  Pr(G=k|X=x) = \frac{exp(\beta_{k0} + \beta_k^Tx)}{1+\sum_{l=1}^{K-1} exp(\beta_{l0} + \beta_l^Tx)}, k=1,...,K-1
\end{equation}

which makes the probability of the class K as follows
\begin{equation}
  Pr(G=k|X=x) = \frac{1}{1+\sum_{l=1}^{K-1} exp(\beta_{l0} + \beta_l^Tx)}
\end{equation}

To simplify notation we would rewrite $P(G=k|X=x)$ as $p_k(x;\theta)$.
The log likelihood of $\theta$ is then
\begin{equation}
  l(\theta) = \sum_{i=1}^N logp_{g_i}(x_i;\theta)
\end{equation}

### **Binary Logistic Regression**


For binary classification, we can encode the classes as 0/1 responses, where $y_i = 1$ whwne $g_i = 1$, and $y_i = 0$ where $g_i = 2$. Let $p_1(x;\theta) = p(x;\theta)$, and $p_2(x;\theta) = 1 - p(x;\theta)$. The log-likelihood can be written as
\begin{align}
  l(\beta) &= \sum_{i=1}^N \{y_ilog(p(x_i;\beta)) + (1-y_i)log(1-p(x_i;\beta))\} \\
  &= \sum_{i=1}^N \{y_i\beta^Tx_i - log(1+exp(\beta^Tx_i))\}
\end{align}

Some calculus properties:
- $\frac{d}{dx} log(f(x)) = \frac{\frac{d}{dx}f(x)}{f(x)}$
- $\frac{d}{dx} exp(f(x)) = f'(x) exp(f(x))$
- Newton method: $x_{k+1} = x_k - \frac{f'(x_k)}{f''(x_k)}$

To minimize the log-likelihood we would have to find the first derivative with respect to \beta to get
\begin{align}
  \frac{\partial l(\beta)}{\partial\beta} &= \sum_{i=1}^N \Big\{y_i\frac{\partial}{\partial\beta}\beta^Tx_i - \frac{\partial}{\partial\beta} log(1 + exp(\beta^Tx_i)) \Big\} \\
  \frac{\partial l(\beta)}{\partial\beta} &= \sum_{i=1}^N \Big\{ y_ix_i - \frac{\frac{\partial}{\partial\beta}(1 + exp(\beta^Tx_i))}{1 + exp(\beta^Tx_i)} \Big\} \\
  \frac{\partial l(\beta)}{\partial\beta} &= \sum_{i=1}^N \Big\{ y_ix_i - \frac{x_iexp(\beta^Tx_i)}{1 + exp(\beta^Tx_i)} \Big\} \\
  \frac{\partial l(\beta)}{\partial\beta} &= \sum_{i=1}^N x_i \Big\{ y_i - \frac{exp(\beta^Tx_i)}{1 + exp(\beta^Tx_i)} \Big\} \\
  \frac{\partial l(\beta)}{\partial\beta} &= \sum_{i=1}^N x_i \big(y_i - p(x_i;\beta) \big) \\
  \frac{\partial l(\beta)}{\partial\beta} &= \mathbf{X}^T(\mathbf{y} - \mathbf{p})
\end{align}

Now for the second derivative we end up with
\begin{align}
  \frac{\partial^2 l(\beta)}{\partial\beta\partial\beta^T} &= -\sum_{i=1}^N x_ix_i^Tp(x_i;\beta)(1-p(x_i;\beta)) \\
  \frac{\partial^2 l(\beta)}{\partial\beta\partial\beta^T} &= -\mathbf{X}^T\mathbf{W}\mathbf{X} \text{ where } \mathbf{W}_{i,j} = p(x_i;\beta)(1-p(x_i;\beta)) \text{ if } i=j \text{, } 0 \text{ otherwise.}
\end{align}

Then we can use Newton method to solve the optimization problem
$$\min_{{\beta \in \mathbb{R}^p}} \hat{y}$$
\begin{align}
  \beta_{k+1} &= \beta_k - \Big(\frac{\partial^2 l(\beta)}{\partial\beta\partial\beta^T}\Big)^{-1}\frac{\partial l(\beta_k)}{\partial \beta_k} \\
  &= \beta_k + (\mathbf{X}^T\mathbf{W}\mathbf{X})^{-1}\mathbf{X}^T(\mathbf{y} - \mathbf{p})
\end{align}

In [ ]:
class BinaryLogisticRegression:
  """
  Moduel use for binary classification task. Optimization use is Newton method.
  ...
  Atributes
  ---------
  self.intercept_ : shape=(1, )
    Real value numbers that represent the bias of each class.
  self.coef_ : shape=(n_attributes, )
    Real value numbers that represent the weights of each class.

   Methods
  ---------
  fit(x_train, y_train)
    Calcualte intercept and beta coefficients using Newtons method.
  predict(x_test)
    Calculate predict values as sigmoid(XB)>0.5.
  """
  def __init__(self, tol: float = 1e-4, max_iter: int = 100) -> None:
    self.tol = tol
    self.max_iter = max_iter

  def fit(self, x_train: NumNPArrayNxM, y_train: ArrayLike, alpha: float = 0.001) -> None:
    """
    Use Newton method for optimization of beta values. Formula to update beta is
    beta_(i+1) = beta_i + (X^T WX)^-1 X^T (y-p)

    Args:
        x_train (NumNPArrayNxM): _description_
        y_train (ArrayLike): _description_
        alpha (float, optional): _description_. Defaults to 0.001.
    """
    #Add a column of 1's at the beginning of the data to calculate y intercept
    X = np.c_[np.ones((x_train.shape[0],1)),x_train]
    #We have as many betas as we have attributes
    betas = np.zeros((X.shape[1],1))
    #Reshape y_train so it is a 1 column matrix
    y = np.reshape(y_train,(-1,1))
    for _ in range(self.max_iter):
      p = sigmoid(np.dot(X,betas))
      W = p*(1-p)
      XTWX = np.matmul(X.T, W*X)
      temp = np.matmul(np.linalg.inv(XTWX),X.T)
      new_betas = betas + np.matmul(temp,(y-p))
      if np.sum(abs(new_betas-betas)) < self.tol:
        break
      betas = new_betas
    self.intercept_ = betas[0,0]
    self.coef_ = betas[1:,0]

  def predict(self, x_test: NumNPArrayNxM) -> NumNPArray:
    """
    Predict y values as sigmoid(XB)>0.5.

    Args:
        x_test (NumNPArrayNxM): _description_

    Returns:
        NumNPArray: _description_
    """
    X = np.c_[np.ones((x_test.shape[0],1)),x_test]
    betas = np.hstack((np.reshape(self.intercept_,(-1)),self.coef_))
    z = np.dot(X,betas)
    return (sigmoid(z)>0.5).astype(int)

  def __str__(self) -> str:
    return '{}(max_iter={:03})'.format(self.__class__.__name__,self.max_iter)

Binary classification test

In [ ]:
from sklearn.linear_model import LogisticRegression as LR

In [ ]:
x_train,x_test,y_train,y_test = process_gamma_dataset()
standardize(x_train)
standardize(x_test)

#My model
print("My model")
t0 = time()
model = BinaryLogisticRegression()
print("model = ", model)
model.fit(x_train, y_train)
pred = model.predict(x_test)
print("Elapse time = {:.3f}".format(time() - t0))
print("accuracy: {:.5f}".format(accuracy(y_test, pred)))

#Run sklearn
print("\nSKLEARN model")
t0 = time()
model = LR(max_iter=100, penalty=None, tol=1e-4)
print("model = ", model)
model.fit(x_train, y_train)
pred = model.predict(x_test)
print("Elapse time = {:.5f}".format(time() - t0))
print("accuracy: {:.5f}".format(accuracy(y_test, pred)))

My model
model =  BinaryLogisticRegression(max_iter=100, penalty=None)
Elapse time = 0.022
accuracy: 0.78759

SKLEARN model
model =  LogisticRegression(penalty=None)
Elapse time = 0.04869
accuracy: 0.78759


### **Multinomial Logistic Regression**


In [ ]:
class LogisticRegression:
  def __P(self, X: NumNPArrayNxM, B: NumNPArray) -> NumNPArray:
    N = X.shape[0]  #Number of object in training set
    P = X.shape[1]
    a = np.empty((N,self.K))
    for i in range(self.K):
      a[:,i] = np.reshape(np.exp(np.dot(X,B[i*P:i*P+P])),(-1))
    a /= np.sum(a,axis=1).reshape((-1,1))
    return np.reshape(a,(-1,1))
    #return np.reshape(a,(-1,1),order='F')

  def __init__(self, tol: float = 1e-4, max_iter: int = 100) -> None:
    self.tol = tol
    self.max_iter = max_iter

  def fit(self, x_train: NumNPArrayNxM, y_train: ArrayLike, alpha: float = 0.001) -> None:
    self.K = np.unique(y_train).shape[0]  #Number of classes
    K = self.K
    #Add a column of 1's at the beginning of the data to calculate y intercept
    X = np.c_[np.ones((x_train.shape[0],1)),x_train]
    N = X.shape[0]  #Number of object in training set
    P = X.shape[1]  #Number of parameters + 1 because of the 1's column
    #Reshape y_train so it is a 1 column matrix
    Y = to_categorical(y_train).reshape((-1,1))
    #Set beta values to 0
    B = np.zeros(shape=(K*P,1))
    new_B = np.empty(shape=(K*P,1))
    for j in range(self.max_iter):
      #Calculate the probabilities
      p = np.empty((N,self.K))
      for i in range(K):
        #p[:,i] = np.reshape(np.exp(np.dot(X,B[i::K])),(-1))
        p[:,i] = np.reshape(np.exp(np.dot(X,B[i*P:i*P+P])),(-1))
      p /= np.sum(p,axis=1).reshape((-1,1))
      p = np.reshape(p,(-1,1))
      #Calculate new betas
      y_p = Y-p
      for i in range(K):
        #W = p[i*N:i*N+N]
        W = p[i::K]                                                                                                                                               
        W = W*(1.-W)
        XTWX = np.matmul(X.T,W*X)
        temp = np.matmul(np.linalg.pinv(XTWX),X.T)
        new_B[i*P:i*P+P] = B[i*P:i*P+P] + np.matmul(temp,y_p[i::K])
        #new_B[i::K] = B[i::K] + np.matmul(temp,y_p[i::K])
        #new_B[i::K] = B[i::K] + np.matmul(temp,y_p[i*N:i*N+N])
        #new_B[i*P:i*P+P] = B[i*P:i*P+P] + np.matmul(temp,y_p[i*N:i*N+N])
      diff = B.reshape((P,K)) - new_B.reshape((P,K))
      print(np.all(np.sum(abs(diff)) < self.tol))
      if np.all(np.sum(abs(diff)) < self.tol):
        print(j)
        break
      B = new_B
    self.intercept_ = np.array(B[0::P])
    #Get all betas that are not the intercepts and store it as a 2D array, 1 row per class coefficients
    self.coef_ = np.delete(B,list(range(0,B.shape[0],P)),axis=0).reshape((-1,P-1))

  def predict(self, x_test: NumNPArrayNxM) -> NumNPArray:
    X = np.c_[np.ones((x_test.shape[0],1)),x_test]
    B = np.hstack((self.intercept_,self.coef_)).reshape((-1))
    p = np.reshape(self.__P(X,B),(-1,self.K))
    return np.argmax(p,axis=1)

  def __str__(self) -> str:
    return '{}(max_iter={:03})'.format(self.__class__.__name__,self.max_iter)

Binary classification with multinomial logistic regression implementation.

In [ ]:
x_train,x_test,y_train,y_test = process_gamma_dataset()
#x_train,x_test,y_train,y_test = mnist()

#standardize(x_train)
#standardize(x_test)

#My model
print("My model")
t0 = time()
model = LogisticRegression()
print("model = ", model)
model.fit(x_train, y_train)
pred = model.predict(x_test)
print("Elapse time = {:.3f}".format(time() - t0))
print("accuracy: {:.5f}".format(accuracy(y_test, pred)))

#Run sklearn simple model
print("\nSKLEARN model")
t0 = time()
model = LR(max_iter=100, penalty=None, tol=1e-4)
print("model = ", model)
model.fit(x_train, y_train)
pred = model.predict(x_test)
print("Elapse time = {:.5f}".format(time() - t0))
print("accuracy: {:.5f}".format(accuracy(y_test, pred)))

My model
model =  LogisticRegression(max_iter=100)
False
True
1
Elapse time = 0.018
accuracy: 0.73843

SKLEARN model
model =  LogisticRegression(penalty=None)
Elapse time = 0.14528
accuracy: 0.79022


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## **Naive Bayes**


Bayes theorem states $$ P(C|X) = \frac{P(X|C)P(C)}{P(X)} $$ which can be use to predict a class C as 
\begin{align}
  \hat{C} &= \underset{c \in C}{\mathrm{argmax}} P(c|X) \\
  &= \underset{c \in C}{\mathrm{argmax}} \frac{P(X|c)P(c)}{P(X)} \\
  &\propto \underset{c \in C}{\mathrm{argmax}} P(X|c)P(c) \\
  &= \underset{c \in C}{\mathrm{argmax}} \Pi_i P(X_i|c)P(c) \\
  &\propto \underset{c \in C}{\mathrm{argmax}} log \Big( P(c) + \sum_i P(X_i|c) \Big)
\end{align}


### **Bernoulli**

For Bernoulli we calculate our conditional probabilities from binarize inputs as $$ P(X_i | y) = P(X_i=1 | y)x_i + (1-P(X_i=1) | y))(1-X_i) $$

In [ ]:
class BernoulliNB:
  """
  Module use to predict y values from binarize X data to calculate conditional
  probabilities as P(y|X) = P(X=1|y)X + (1-P(X=1|y))(1-X).
  ...
  Atributes
  ---------
  self.binarize : float
    Value use to binarize data. If None is given we assume data is already
    binarized.
  self.alpha : float
    Value use as smoother in the conditional probabilities.
  self.classes_ : shape=(n_classes, )
    Numpy array with all classes sorted in ascending order.
  self.pac1 : shape = (n_classes, n_attributes, )
    2D numpy array with the conditional probabilities of such aattribute in
    class being equal to 1.
  self.pac0 : shape = (n_classes, n_attributes, )
    Same as self.pac1 but for value 0.
  self.class_prior_ : shape = (n_classes, )
    Log probability for getting a class.

   Methods
  ---------
  fit(x_train, y_train)
    Calcualte the mean occurance of each class and the probability of each
    attribute beeing 1 or zero given the class.
  predict(x_test)
    Using probability of each class and the the prbability of 1 or 0 for each
    attribute in each class we predict using the following formula:
    P(X|y) = P(X=1|y)X + (1-P(X=1|y))(1-X).
  """
  def __init__(self, alpha: float = 1.0, force_alpha: bool = False, binarize: float | None = None) -> None:
    self.binarize = binarize
    if alpha is not None:
      self.alpha = 1e-10 if force_alpha and alpha<1e-10 else alpha
    else:
      self.alpha = -1.0

  def fit(self, x_train: NumNPArrayNxM, y_train: ArrayLike) -> None:
    """
    Calcate the mean occurance of each class as np.mean(y_train==y) and then take
    the log probability of each to calculate self.class_prior_. Also, calculate
    self.pac1[i] as np.mean(x_train[y_train==y]) for each attribute.
    """
    if self.binarize is not None:
      x_train = (x_train>self.binarize).astype(int)

    self.classes_ = np.sort(np.unique(y_train))
    pc = np.empty(self.classes_.shape[0])
    self.pac1 = np.empty((self.classes_.shape[0],x_train.shape[1]))

    for i,y in enumerate(self.classes_):
      # Identify which sample match this class
      ind = y_train==y
      # Calculate probability to get 1
      self.pac1[i] = np.mean(x_train[ind],axis=0)
      # Calculate average of how many samples match this class
      pc[i] = np.mean(ind)

    #Add smoothing and re-normalize probabilities
    if self.alpha > 0.0:
      pc += self.alpha # Add smoothing
      self.pac1 = self.pac1*(1-2*self.alpha) + self.alpha
    self.pac0 = 1-self.pac1
    self.class_prior_ = np.log(pc)

  def predict(self, x_test: NumNPArrayNxM) -> ArrayLike:
    """
    We predict each class probability as 𝑙𝑜𝑔(𝑃(y)+∑𝑃(𝑋𝑖|y) where 𝑙𝑜𝑔(𝑃(y) is
    self.class_prior_ and 𝑃(𝑋𝑖|y) = P(X=1|y)X + (1-P(X=1|y))(1-X).
    """
    if self.binarize is not None:
      x_test = (x_test>self.binarize).astype(int)

    x_test_0 = 1.0-x_test
    probs = np.empty((x_test.shape[0],self.classes_.shape[0]))
    for i in range(self.classes_.shape[0]):
      p = self.pac1[i:i+1]*x_test + self.pac0[i:i+1]*x_test_0
      probs[:,i] = np.sum(np.log(p),axis=1)
    return self.classes_[np.argmax(probs + self.class_prior_,axis=1)]

Classification test

In [ ]:
from sklearn.naive_bayes import BernoulliNB as BNB

In [ ]:
x_train,x_test,y_train,y_test = mnist_x_train, mnist_x_test, mnist_y_train, mnist_y_test
mean = np.mean(x_train)

print("My model:")
t0 = time()
model = BernoulliNB(alpha=1e-6,binarize=mean)
model.fit(x_train,y_train)
pred = model.predict(x_test)
print("Elapse time = {:.5f}".format(time() - t0))
print("accuracy: {:.5f}".format(accuracy(y_test, pred)))

print("\nSKLEARN model:")
t0 = time()
model = BNB(alpha=1e-6,binarize=mean)
model.fit(x_train,y_train)
pred = model.predict(x_test)
print("Elapse time = {:.5f}".format(time() - t0))
print("accuracy: {:.5f}".format(accuracy(y_test, pred)))

My model:
Elapse time = 2.87926
accuracy: 0.84650

SKLEARN model:
Elapse time = 1.10330
accuracy: 0.84700


### **Gaussian**

We calculate the conditional probabilities as $$ P(X_i|y) = \frac{1}{\sigma_y\sqrt{2\pi}} exp\bigg( -\frac{(X_i-\mu_y)^2}{2\sigma_y^2} \bigg)$$

In [ ]:
class GaussianNB:
  """
  Model to predict y values by assuming that all attributes are normaly
  distributed. We can calculate probabilities as
  P(X|y)=1/(sigma*sqrt(2pi))*exp(-(X-mu_y)^2/2sigma^2)
  ...
  Atributes
  ---------
  self.epsilon_ : float
    Smoother use on each self.var_ entry to avoid numerical error calculations.
  self.classes_ : (n_classes, )
    Numpy array with all classes sorted in ascending order.
  self.var_ : (n_classes, n_attributes, )
    Variance of each attribute for each class.
  self.theta_ : (n_classes, n_attributes, )
    Mean of each attribute for each class.
  self.class_prior_ : (n_classes, )
    Log probability for getting a class.

   Methods
  ---------
  fit(x_train, y_train)
    Calculate the mean occurance of each class, the variance and mean of each
    attribute for each class.
  predict(x_test)
    Use gauss distribution assumption it calculate the conditional probability
    and take the maximum argument from all probabilities.
  """
  def __init__(self, var_smoothing: float = 1e-09) -> None:
    self.epsilon_ = var_smoothing

  def fit(self, x_train: NumNPArrayNxM, y_train: ArrayLike) -> None:
    """
    Get all possible predict values, for each class it calculate the variance
    and mean of each attribute.

    Args:
        x_train (NumNPArrayNxM): _description_
        y_train (ArrayLike): _description_
    """
    self.classes_ = np.sort(np.unique(y_train))
    self.var_ = np.empty((self.classes_.shape[0],x_train.shape[1]))
    self.theta_ = np.empty((self.classes_.shape[0],x_train.shape[1]))
    pc = np.empty(self.classes_.shape[0])

    for i,y in enumerate(self.classes_):
      # Identify which sample match this class
      ind = y_train==y
      sub_x = x_train[ind]
      self.theta_[i] = np.mean(sub_x,axis=0)
      self.var_[i] = np.std(sub_x,axis=0) + self.epsilon_
      pc[i] = np.mean(ind)
    self.class_prior_ = np.log(pc)

  def predict(self, x_train: NumNPArrayNxM) -> ArrayLike:
    """
    We predict each class probability as log(P(y)+∑𝑃(Xi|y) where log(P(y) is
    self.class_prior_ and P(Xi|y) = 1/(sigma*sqrt(2pi))*exp(-(X-mu_y)^2/2sigma^2)

    Args:
        x_train (NumNPArrayNxM): _description_

    Returns:
        ArrayLike: _description_
    """
    np.seterr(divide = 'ignore')
    probs = np.empty((x_test.shape[0],self.classes_.shape[0]))
    t = 1.0/np.sqrt(2.0*pi)
    for i in range(self.classes_.shape[0]):
      p = (t/self.var_[i])*np.exp(-0.5*((x_test-self.theta_[i])/self.var_[i])**2)
      probs[:,i] = np.sum(np.log(p),axis=1)
    return self.classes_[np.argmax(probs + self.class_prior_,axis=1)]

Classification test

In [ ]:
from sklearn.naive_bayes import GaussianNB as GNB

In [ ]:
x_train,x_test,y_train,y_test = mnist_x_train, mnist_x_test, mnist_y_train, mnist_y_test

print("My model:")
t0 = time()
model = GaussianNB(var_smoothing=1e-6)
model.fit(x_train,y_train)
pred = model.predict(x_test)
print("Elapse time = {:.5f}".format(time() - t0))
print("accuracy: {:.5f}".format(accuracy(y_test, pred)))

print("\nSKLEARN model:")
t0 = time()
model = GNB(var_smoothing=1e-6)
model.fit(x_train,y_train)
pred = model.predict(x_test)
print("Elapse time = {:.5f}".format(time() - t0))
print("accuracy: {:.5f}".format(accuracy(y_test, pred)))

My model:
Elapse time = 3.60504
accuracy: 0.52710

SKLEARN model:
Elapse time = 1.25368
accuracy: 0.61510


## **Decision Trees**


In [ ]:
def best_feature_DT(x_train: NumNPArrayNxM, y_train: ArrayLike) -> int:
  best_feature = -1
  best_acc = -1.0
  for i_feature in range(x_train.shape[1]):
    sub_x = x_train[:,i_feature]
    if isinstance(sub_x[0], (np.integer, np.floating)):
      i_mean = np.mean(sub_x)
      l_temp = calc_predict(y_train[sub_x<=i_mean])
      r_temp = calc_predict(y_train[sub_x>i_mean])
      curr_acc = (l_temp[1] + r_temp[1])/y_train.shape[0]
    else:
      curr_acc = 0.0 
      for option in np.unique(sub_x):
        curr_acc += calc_predict(y_train[sub_x==option])[1]
      curr_acc /= y_train.shape[0]

    if best_acc < curr_acc:
      best_acc = curr_acc
      best_feature = i_feature
  return best_feature

def calc_predict(y_train: ArrayLike) -> Tuple[int, int]:
  #Get all possible values of y_train

  #Calculate which options gives the highest accuracy
  largest_sum = -1
  best_option = -1
  for option in np.unique(y_train):
    curr_sum = np.sum(y_train==option)
    if largest_sum < curr_sum:
      largest_sum = curr_sum
      best_option = option
  return (best_option, largest_sum)

class InvalidParameterError(Exception):
  def __init__(self, messate: str = "Invalid parameter") -> None:
    print(messate)

class DecisionTreeClassifier:
  def __init__(self, splitter: str = "best", max_depth: int | None = None) -> None:
    #Decision tree cannot be a leaf unless it have a predicted value assigned
    self.is_leaf = False
    self.splitter = splitter
    self.max_depth = max_depth

  def split_feature_DT(self, x_train: NumNPArrayNxM, y_train: ArrayLike, feature_idx: int) -> None:
    self.branches = dict()
    self.feature_idx = feature_idx
    #Get all rows for feature selected
    sub_x = x_train[:,feature_idx]
    #If the attribute is numeric we use the mean, if it is categorical we use the unique values
    if isinstance(sub_x[0], (np.integer, np.floating)):
      self.isnumeric = True
      #Split using the mean
      self.break_point = np.mean(sub_x)
      for i in range(2):
        new_depth = None if self.max_depth is None else self.max_depth-1
        dt = DecisionTreeClassifier(splitter=self.splitter, max_depth=new_depth)
        #Add dt into the tree and get the indices for the left or right branch
        if i == 0:
          new_idxs = sub_x <= self.break_point
          self.branches["le"] = dt
        else:
          new_idxs = sub_x > self.break_point
          self.branches["g"] = dt
        #If max_depth is 1 or if only one y value is left, we create the leafs
        new_y = y_train[new_idxs]
        if self.max_depth == 1 or np.unique(new_y).shape[0]==1:
          #Set dt as a leaf and fill in its attributes
          dt.is_leaf = True
          dt.pred_val = calc_predict(new_y)[0]
        else:
          dt.fit(x_train[new_idxs], new_y)
    #If the attribute is categorical we use its different possible values
    else:
      self.isnumeric = False
      #Get all possible classification values from sub_x
      options = np.unique(sub_x)
      #Make a branch for each option
      for option in options:
        new_depth = None if self.max_depth is None else self.max_depth-1
        dt = DecisionTreeClassifier(splitter=self.splitter, max_depth=new_depth)
        #Add dt as a branch                                                                                                                                                           
        self.branches[option] = dt
        new_idxs = sub_x==option
        new_y = y_train[new_idxs]
        #If max_depth is 1 or if only one y value is left, we create the leafs
        if self.max_depth == 1 or np.unique(new_y).shape[0]==1:
          #Set dt as a leaf and fill in its attributes
          dt.is_leaf = True
          dt.pred_val = calc_predict(new_y)[0]
        else:
          dt.fit(x_train[new_idxs], new_y)

  def fit(self, x_train: NumNPArrayNxM, y_train: ArrayLike, feature_idx: int | None = None) -> None:
    #If max_depth is less than 1 we fail
    if self.max_depth is not None and self.max_depth < 1:
      raise InvalidParameterError("max_depth must be in the range [1,inf)")

    #feature_idx is out of bound we fail
    if feature_idx is not None and (feature_idx < 0 or feature_idx > x_train.shape[1]):
      raise InvalidParameterError(f"feature_idx must be in the range [0,{x_train.shape[1]}]")

    #Use feature index given or use splitter option to get one
    if (feature_idx is None):
      if self.splitter == "best":
        best_feature = best_feature_DT(x_train, y_train)
        self.split_feature_DT(x_train, y_train, best_feature)
      elif self.splitter == "random":
        self.split_feature_DT(x_train, y_train, np.random.randint(0, x_train.shape[1], 1)[0])
    #Else use the feature column that is given
    else:
      self.split_feature_DT(x_train, y_train, feature_idx)

  def predict(self, x_train: NumNPArrayNxM) -> ArrayLike:
    #Make prediction array
    pred = np.empty(shape=(x_test.shape[0]))

    if self.is_leaf:
      pred.fill(self.pred_val)
      return pred

    #Create sub_x with the column of interest
    sub_x = x_test[:,self.feature_idx]

    #For this simple case we only have leafs
    if self.isnumeric:
      #Do a recursive prediction for both branches
      l_sub = sub_x<=self.break_point
      pred[l_sub] = self.branches["le"].predict(x_test[l_sub])
      r_sub = sub_x > self.break_point
      pred[r_sub] = self.branches["g"].predict(x_test[r_sub])
    else:                                                                                                                                                                             
      for option,dt in self.branches.items():
        match_x = sub_x==option
        #Select x_test object that match this branch and assign prediction values
        pred[match_x] = dt.predict(x_test[match_x])
    return pred

Classification test

In [ ]:
from sklearn.tree import DecisionTreeClassifier as DTC

In [ ]:
x_train, x_test, y_train, y_test = mnist_x_train, mnist_x_test, mnist_y_train, mnist_y_test

print("My model")
t0 = time()
model = DecisionTreeClassifier(splitter="best", max_depth=10)
model.fit(x_train, y_train)
pred = model.predict(x_test)
print("Elapse time = {:.3f}".format(time() - t0))
print("a: model accuracy = ", accuracy(y_test, pred))

print("SKLEARN model:")
t0 = time()
model = DTC(splitter="best", max_depth=10)
model.fit(x_train, y_train)
pred = model.predict(x_test)
print("Elapse time = {:.3f}".format(time() - t0))
print("a: model accuracy = ", accuracy(y_test, pred))

My model
Elapse time = 112.840
a: model accuracy =  0.8294
SKLEARN model:
Elapse time = 12.224
a: model accuracy =  0.8654


## **Dense Neural Network**


In [ ]:
class MLPRegressor:
  def __init__(
    self,
    hidden_layer_sizes: Sequence[int] = (100,),
    activation: str = 'relu',
    solver: str = 'sgd',
    learning_rate_init: float = 0.001,
    momentum: float = 0.9
  ) -> None:
    self.hidden_layer_sizes = hidden_layer_sizes
    self.act = activation
    self.solv = solver
    self.lern_rate = learning_rate_init
    self.momentum = momentum

  def fit(self, x_train: NumNPArrayNxM, y_train: ArrayLike) -> None:
    #Create spaces for weights and biasses
    self.coef_ = [np.random.randn(x_train.shape[1],self.hidden_layer_sizes[0])*np.sqrt(2./self.hidden_layer_sizes[0])]
    self.intercepts_ = [np.zeros((self.hidden_layer_sizes[0]))]
    change_W = [np.zeros(shape=self.coef_[0].shape)]
    change_b = [np.zeros(shape=self.intercepts_[0].shape)]
    for size in self.hidden_layer_sizes[1:]:
      self.coef_.append(np.random.randn(self.coef_[-1].shape[1],size)*np.sqrt(2./self.hidden_layer_sizes[0]))
      self.intercepts_.append(np.zeros((size)))
      change_W.append(np.zeros(self.coef_[-1].shape))
      change_b.append(np.zeros((size)))
    self.coef_.append(np.random.randn(self.coef_[-1].shape[1],1)*np.sqrt(2.))
    self.intercepts_.append(np.zeros((1)))
    change_W.append(np.zeros(self.coef_[-1].shape))
    change_b.append(np.zeros(1))

    #Calculate batch_size
    batch_size = min(200, x_train.shape[0])
    n_batches = y_train.shape[0]//batch_size

    for i_batch in range(n_batches):
      Z = [0]*len(self.coef_)
      A = [0]*len(self.coef_)
      dZ = [0]*len(self.coef_)
      dW = [0]*len(self.coef_)
      db = [0]*len(self.coef_)

      #Subset x_train and y_train for this batch
      start_row = i_batch*batch_size
      sub_x = x_train[start_row:start_row+batch_size]
      sub_y = y_train[start_row:start_row+batch_size].reshape((-1,1))

      #Forward propagation
      Z[0] = np.matmul(sub_x,self.coef_[0]) + self.intercepts_[0]
      A[0] = relu(Z[0])
      for i in range(1,len(Z)):
        Z[i] = np.matmul(A[i-1],self.coef_[i]) + self.intercepts_[i]                                                                                                                  
        A[i] = relu(Z[i])

      #Backward propagation
      i = len(Z)-1
      dZ[i] = A[i] - sub_y
      dW[i] = np.matmul(A[i-1].T,dZ[i],)/batch_size
      db[i] = np.sum(dZ[i],axis=1,keepdims=True)/batch_size
      for i in range(len(Z)-2,0,-1):
        dZ[i] = np.matmul(dW[i+1].T,dZ[i+1]) * relu(Z[i])
        dW[i] = np.matmul(dZ[i],A[i].T)/batch_size
        db[i] = np.sum(dZ[i],axis=1,keepdims=True)/batch_size
      dZ[0] = np.matmul(dZ[1],dW[1].T) * relu(Z[0])
      dW[0] = np.matmul(sub_x.T,dZ[0])/batch_size
      db = np.sum(dZ[0],axis=1,keepdims=True)/batch_size

      #Update weights and biases
      for i in range(len(Z)):
        change_W[i] = self.lern_rate*dW[i] + self.momentum*change_W[i]
        self.coef_[i] -= change_W[i]
        change_b[i] = self.lern_rate*db[i] + self.momentum*change_b[i]
        self.intercepts_[i] -= change_b[i]

  def predict(self, x_test: NumNPArrayNxM) -> ArrayLike:
    H = np.matmul(x_test,self.coef_[0]) + self.intercepts_[0]
    for i in range(1,len(self.coef_)):
      H = relu(H)
      H = np.matmul(H,self.coef_[i]) + self.intercepts_[i]
    return H

Regression task

In [ ]:
from sklearn.neural_network import MLPRegressor as MLPR

In [ ]:
x_train,x_test,y_train,y_test = process_gpu_running_time()

print("My model:")
t0 = time()
model = MLPRegressor()
model.fit(x_train,y_train)
pred = model.predict(x_test)
print("Elapse time = {:.5f}".format(time() - t0))
print("MSE = ", MSE(y_test, pred))

print("\nSKLEARN model:")
t0 = time()
model = MLPR(hidden_layer_sizes=(100,),activation='relu',solver='sgd',learning_rate_init=0.001,momentum=0.9)
model.fit(x_train,y_train)
pred = model.predict(x_test)
print("Elapse time = {:.5f}".format(time() - t0))
print("MSE = ", MSE(y_test, pred))